# Reproducing Parameter Estimations

This notebook demonstrates the steps to reproduce parameter estimations for enzyme cascade reactions. The workflow includes data loading, calibration, activity calculation, and parameter fitting.

In [9]:
# Import required libraries
import os
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

from data_hadler import calculate_calibration, get_rates_and_concentrations
from parameter_estimator import estimate_parameters

In [10]:
# Define paths and load calibration data
BASE_PATH = r"C:\\Users\\berger\\Documents\\Projekts\\enzyme-cascade-analysis\\Fehlerfortpflanzunganalyse"

calibration_data = pd.read_csv(os.path.join(BASE_PATH, 'Data', 'NADH_Kalibriergerade.csv'))
calibration_slope = calculate_calibration(calibration_data)
print(f"Calibration slope: {calibration_slope}")

Kalibrierung: Steigung = 3.00, R² = 0.9994
Calibration slope: 2.996680555555556


In [11]:
# Load reaction data and calculate rates and concentrations
r1_path = os.path.join(BASE_PATH, 'Data', 'Reaction1')
r1_nad_data = pd.read_csv(os.path.join(r1_path, 'r_1_NAD_PD_500mM.csv'), header=None)
r1_pd_data = pd.read_csv(os.path.join(r1_path, 'r1_PD_NAD_5mM.csv'), header=None)

reac1_activity_param = {
    "Vf_well": 10.0,
    "Vf_prod": 1.0,
    "c_prod": 2.2108
}

reac_1_rates = {
    "r1_nad_rates": get_rates_and_concentrations(r1_nad_data, calibration_slope, reac1_activity_param)[0],
    "r1_pd_rates":  get_rates_and_concentrations(r1_pd_data, calibration_slope, reac1_activity_param)[0],
}

reac_1_concentrations = {
    "r1_nad_conc": get_rates_and_concentrations(r1_nad_data, calibration_slope, reac1_activity_param)[1],
    "r1_nad_const": 5.0,  # Konstante NAD Konzentration
    "r1_pd_conc":  get_rates_and_concentrations(r1_pd_data, calibration_slope, reac1_activity_param)[1],
    "r1_pd_const": 500.0,  # Konstante PD Konzentration
}

print("Reaction 1 NAD Rates:", reac_1_rates["r1_nad_rates"])
print("Reaction 1 PD Rates:", reac_1_rates["r1_pd_rates"])

Well 1 (Konz: 0.0 mM): R² = 0.092 - nicht linear genug
Well 2 (Konz: 0.0 mM): R² = 0.097 - nicht linear genug
Well 3: 1.00 mM → Aktivität: 0.022898 U/mg
Well 4: 1.00 mM → Aktivität: 0.021928 U/mg
Well 5: 2.00 mM → Aktivität: 0.031797 U/mg
Well 6: 2.00 mM → Aktivität: 0.034679 U/mg
Well 7: 3.00 mM → Aktivität: 0.038776 U/mg
Well 8: 3.00 mM → Aktivität: 0.039649 U/mg
Well 9: 4.00 mM → Aktivität: 0.042043 U/mg
Well 10: 4.00 mM → Aktivität: 0.045990 U/mg
Well 11: 5.00 mM → Aktivität: 0.050625 U/mg
Well 12: 5.00 mM → Aktivität: 0.050075 U/mg
Well 13: 6.00 mM → Aktivität: 0.052413 U/mg
Well 14: 6.00 mM → Aktivität: 0.050983 U/mg
Well 15: 7.00 mM → Aktivität: 0.050328 U/mg
Well 16: 7.00 mM → Aktivität: 0.053309 U/mg
Erfolgreich 14 gültige Datenpunkte verarbeitet
Well 1 (Konz: 0.0 mM): R² = 0.506 - nicht linear genug
Well 2 (Konz: 0.0 mM): R² = 0.382 - nicht linear genug
Well 3 (Konz: 25.0 mM): R² = 0.860 - nicht linear genug
Well 4 (Konz: 25.0 mM): R² = 0.788 - nicht linear genug
Well 5: 50.0

In [12]:
# Define the model and estimate parameters
def two_substrat_michaelis_menten(concentration_data, Vmax, Km1, Km2): 
    """Zwei-Substrat Michaelis-Menten Gleichung"""
    S1_values, S2_values = concentration_data

    # Sicherstellen, dass es numpy arrays sind
    S1_values = np.asarray(S1_values)
    S2_values = np.asarray(S2_values)
    
    # Element-weise Berechnung für alle Datenpunkte
    rates = (Vmax * S1_values * S2_values) / ((Km1 + S1_values) * (Km2 + S2_values))
    
    return rates

reac_1_model = {
    "name": "two_substrat_michaelis_menten",
    "function": two_substrat_michaelis_menten,
    "param_names": ["Vmax", "Km_NAD", "Km_PD"],
    "param_units": ["U", "mM", "mM"],
    "substrate_keys": ["r1_nad_conc", "r1_pd_conc"],
    "initial_guess_func": lambda activities, substrate_data: [max(activities), 1.0, 1.0],
    "bounds_lower": [0, 0, 0],
    "bounds_upper": [np.inf, np.inf, np.inf],
    "description": "Zwei-Substrat Michaelis-Menten für Reaktion 1"
}

reac_1_parameters = estimate_parameters(reac_1_model, reac_1_concentrations, reac_1_rates)

print("\n=== Reaktion 1 Parameter Schätzung ==="
          f"\nModell: {reac_1_model['description']}"
          f"\nErgebnis: {reac_1_parameters}"
          f"\nR²: {reac_1_parameters['r_squared']:.4f}"
          f"\nVmax: {reac_1_parameters['params'][0]:.4f} {reac_1_model['param_units'][0]}"
          f"\nKm1: {reac_1_parameters['params'][1]:.4f} {reac_1_model['param_units'][1]}"
          f"\nKm2: {reac_1_parameters['params'][2]:.4f} {reac_1_model['param_units'][2]}"
          f"\nVmax Fehler: {reac_1_parameters['param_errors'][0]:.4f} {reac_1_model['param_units'][0]}"
          f"\nKm1 Fehler: {reac_1_parameters['param_errors'][1]:.4f} {reac_1_model['param_units'][1]}"
          f"\nKm2 Fehler: {reac_1_parameters['param_errors'][2]:.4f} {reac_1_model['param_units'][2]}"
            )


Debug - Total activities: 26
Debug - Multi-Substrat-Modell mit 2 Substraten
Debug - Suche nach: var='r1_nad_conc', const='r1_nad_const'
Debug - Suche nach: var='r1_pd_conc', const='r1_pd_const'
Debug - Variable Daten: 2
Debug - Konstante Werte: 2
Debug - r1_nad_conc variiert: 14 Punkte
Debug - r1_pd_conc variiert: 12 Punkte
Debug - r1_nad_conc: shape (26,), first 5 values: [1. 1. 2. 2. 3.]
Debug - r1_nad_conc: unique values: [1. 2. 3. 4. 5. 6. 7.]
Debug - r1_pd_conc: shape (26,), first 5 values: [500. 500. 500. 500. 500.]
Debug - r1_pd_conc: unique values: [ 50.  75. 100. 200. 350. 500. 650.]
Initial guess: [np.float64(0.05330916209607437), 1.0, 1.0]

=== Reaktion 1 Parameter Schätzung ===
Modell: Zwei-Substrat Michaelis-Menten für Reaktion 1
Ergebnis: {'success': True, 'params': array([7.74413089e-02, 1.93540939e+00, 1.01810991e+02]), 'param_errors': array([4.89151331e-03, 3.34588769e-01, 1.29390003e+01]), 'r_squared': np.float64(0.9359502024467375), 'model_name': 'two_substrat_michae